In [9]:
import os
import sys
import yaml
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from glob import glob
from sklearn.model_selection import train_test_split
from collections import OrderedDict
from tqdm import tqdm

# --- 1. ADD YOUR PROJECT'S 'source' FOLDER TO THE PYTHON PATH ---

# Get the project root directory (which contains 'Code' and 'Medical Image segmentation Notebook')
# os.getcwd() is '.../Medical Image segmentation Notebook'
# os.path.abspath(os.path.join(os.getcwd(), "..")) is '.../Medical-Image-Segmentation-Deep-Learning-Project'
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

# This is the folder containing all your .py modules (ML_Pipeline, engine.py, etc.)
SOURCE_DIR = os.path.join(PROJECT_ROOT, "Code", "source")

# Add this to sys.path so Python can find your modules
if SOURCE_DIR not in sys.path:
    sys.path.append(SOURCE_DIR)

print(f"✅ Project Root: {PROJECT_ROOT}")
print(f"✅ Source Dir added to path: {SOURCE_DIR}")


# --- 2. IMPORT FROM YOUR .py FILES ---
# (Now that the path is set, these imports will work)

from ML_Pipeline.dataset import DataSet
from ML_Pipeline.network import UNetPP
from ML_Pipeline.utils import AverageMeter, iou_score
from ML_Pipeline.train import train, validate  # Make sure your train.py has both

# --- 3. IMPORT AUGMENTATIONS ---
from albumentations import (
    Compose, OneOf, RandomRotate90, Flip, HueSaturationValue,
    RandomBrightnessContrast, Resize, Normalize
)
from albumentations.pytorch import ToTensorV2

✅ Project Root: c:\Users\hp\Desktop\image_Segmentation\Medical-Image-Segmentation-Deep-Learning-Project
✅ Source Dir added to path: c:\Users\hp\Desktop\image_Segmentation\Medical-Image-Segmentation-Deep-Learning-Project\Code\source


In [10]:
# Load the config file
config_path = os.path.join(SOURCE_DIR, "config.yaml")
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

# --- Resolve all paths from config ---
# (Uses PROJECT_ROOT from Cell 1 to build correct absolute paths)
config["image_path"] = os.path.join(PROJECT_ROOT, config["image_path"])
config["mask_path"] = os.path.join(PROJECT_ROOT, config["mask_path"])
config["model_path"] = os.path.join(PROJECT_ROOT, config["model_path"])
config["log_path"] = os.path.join(PROJECT_ROOT, config["log_path"])
config["output_path"] = os.path.join(PROJECT_ROOT, config["output_path"])

# --- Print to verify ---
print("🔧 Config loaded and paths resolved:")
print(f"  Image Path: {config['image_path']}")
print(f"  Mask Path: {config['mask_path']}")
print(f"  Model Path: {config['model_path']}")

🔧 Config loaded and paths resolved:
  Image Path: c:\Users\hp\Desktop\image_Segmentation\Medical-Image-Segmentation-Deep-Learning-Project\Code/input/PNG/Original
  Mask Path: c:\Users\hp\Desktop\image_Segmentation\Medical-Image-Segmentation-Deep-Learning-Project\Code/input/PNG/Ground Truth
  Model Path: c:\Users\hp\Desktop\image_Segmentation\Medical-Image-Segmentation-Deep-Learning-Project\Code/output/models/model.pth


In [3]:
# Get all image file names (without extension)
img_ids = glob(os.path.join(config["image_path"], f"*{config['extn']}"))
img_ids = [os.path.splitext(os.path.basename(p))[0] for p in img_ids]

if not img_ids:
    print(f"❌ ERROR: No images found at {config['image_path']}")
else:
    # Split into train and validation sets
    train_img_ids, val_img_ids = train_test_split(img_ids, test_size=0.2, random_state=42)
    print(f"✅ Found {len(img_ids)} images. Split into {len(train_img_ids)} train, {len(val_img_ids)} val.")

# Define augmentations
train_transform = Compose([
    RandomRotate90(),
    Flip(),
    OneOf([
        HueSaturationValue(),
        RandomBrightnessContrast(),
    ], p=1),
    Resize(256, 256),
    Normalize(),
    ToTensorV2(),  # Convert numpy array to torch.Tensor
])

val_transform = Compose([
    Resize(256, 256),
    Normalize(),
    ToTensorV2(),  # Convert numpy array to torch.Tensor
])

✅ Found 612 images. Split into 489 train, 123 val.


In [4]:
# Create Dataset objects (using the DataSet class imported from ML_Pipeline/dataset.py)
train_dataset = DataSet(
    img_ids=train_img_ids,
    image_path=config["image_path"],  # Use the absolute path
    mask_path=config["mask_path"],    # Use the absolute path
    img_ext=config["extn"],
    mask_ext=config["extn"],
    transform=train_transform
)

val_dataset = DataSet(
    img_ids=val_img_ids,
    image_path=config["image_path"],
    mask_path=config["mask_path"],
    img_ext=config["extn"],
    mask_ext=config["extn"],
    transform=val_transform
)

# Create DataLoader objects
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.get("batch_size", 4),
    shuffle=True,
    drop_last=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=config.get("batch_size", 4),
    shuffle=False,
    drop_last=False
)

print("✅ DataLoaders created.")

✅ DataLoaders created.


In [5]:
print("--- 🐛 DEBUGGING DATA SHAPES 🐛 ---")

# Get exactly one batch of data from the loader
debug_images, debug_masks, _ = next(iter(train_loader))

print(f"Image batch shape: {debug_images.shape}")
print(f"Mask batch shape:  {debug_masks.shape}")

print("\n--- EXPECTED SHAPES ---")
print("Image shape should be: torch.Size([4, 3, 256, 256])")
print("Mask shape should be:  torch.Size([4, 1, 256, 256])")

--- 🐛 DEBUGGING DATA SHAPES 🐛 ---
Image batch shape: torch.Size([4, 3, 256, 256])
Mask batch shape:  torch.Size([4, 256, 256, 1])

--- EXPECTED SHAPES ---
Image shape should be: torch.Size([4, 3, 256, 256])
Mask shape should be:  torch.Size([4, 1, 256, 256])


In [6]:
# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create model (from ML_Pipeline/network.py)
model = UNetPP(num_classes=1, deep_supervision=True)
model = model.to(device)

# Define Loss Function
criterion = nn.BCEWithLogitsLoss()

# Define Optimizer
params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = optim.Adam(params, lr=config.get("learning_rate", 1e-4), weight_decay=1e-4)

print("✅ Model, Criterion, and Optimizer initialized.")

Using device: cpu
✅ Model, Criterion, and Optimizer initialized.


In [8]:
log = OrderedDict([
    ('epoch', []), ('loss', []), ('iou', []),
    ('val_loss', []), ('val_iou', []),
])

best_iou = 0
trigger = 0
epochs = config.get("epochs", 3)  # Get epoch count from config, default to 10
patience = config.get("early_stopping_patience", 15)

for epoch in range(epochs):
    print(f"\n--- Epoch [{epoch+1}/{epochs}] ---")

    # Train for one epoch (using imported 'train' function)
    train_log = train(True, train_loader, model, criterion, optimizer, device)
    
    # Validate (using imported 'validate' function)
    val_log = validate(True, val_loader, model, criterion, device)

    # Print logs
    print(f"Epoch {epoch+1} | Train Loss: {train_log['loss']:.4f} | Train IoU: {train_log['iou']:.4f} | Val Loss: {val_log['loss']:.4f} | Val IoU: {val_log['iou']:.4f}")

    # Update log dictionary
    log['epoch'].append(epoch)
    log['loss'].append(train_log['loss'])
    log['iou'].append(train_log['iou'])
    log['val_loss'].append(val_log['loss'])
    log['val_iou'].append(val_log['iou'])

    # Save log to CSV
    pd.DataFrame(log).to_csv(config['log_path'], index=False)

    # Early stopping and model checkpointing
    trigger += 1
    if val_log['iou'] > best_iou:
        torch.save(model.state_dict(), config['model_path'])
        print(f"✅ Model Saved! IoU improved from {best_iou:.4f} to {val_log['iou']:.4f}")
        best_iou = val_log['iou']
        trigger = 0
    
    if trigger >= patience:
        print(f"⚠️ Early stopping! No improvement in {patience} epochs.")
        break

print("\n🎉 Training complete!")


--- Epoch [1/300] ---


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Load the trained model ---
# (Make sure to import image_loader from your predict.py file)
from ML_Pipeline.predict import image_loader 

model.load_state_dict(torch.load(config['model_path']))
model.eval()
print("✅ Best model loaded for prediction.")

# --- 2. Load a test image ---
# This path is relative to your PROJECT_ROOT
test_img_name = "115.png" # You can change this to any image name
test_img_path = os.path.join(PROJECT_ROOT, "Code", "input", "PNG", "Original", test_img_name)

if not os.path.exists(test_img_path):
    print(f"❌ Test image not found at: {test_img_path}")
else:
    # Use the image_loader from predict.py
    # It applies Resize, Normalize, and ToTensorV2
    image = image_loader(test_img_path, val_transform) # Pass the validation transform
    
    # Add batch dimension (B, C, H, W) and send to device
    image = image.unsqueeze(0).to(device) 

    # --- 3. Make prediction ---
    with torch.no_grad():
        output = model(image)
        
    # Get the last output if deep supervision is on
    if isinstance(output, list):
        output = output[-1]
        
    # --- 4. Process and show the mask ---
    # Apply sigmoid, remove batch dim, send to CPU, convert to numpy
    mask = torch.sigmoid(output).squeeze().cpu().numpy()
    
    # Convert probabilities (0.0 to 1.0) to a binary mask (0 or 255)
    mask_binary = (mask > 0.5).astype(np.uint8) * 255

    # --- 5. Show Original, Predicted Mask, and Ground Truth ---
    gt_mask_path = os.path.join(PROJECT_ROOT, "Code", "input", "PNG", "Ground Truth", test_img_name)
    gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
    original_image = cv2.imread(test_img_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB) # for plt

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(original_image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")
    
    ax[1].imshow(mask_binary, cmap="gray")
    ax[1].set_title("Predicted Mask")
    ax[1].axis("off")
    
    ax[2].imshow(gt_mask, cmap="gray")
    ax[2].set_title("Ground Truth")
    ax[2].axis("off")
    
    plt.show()
    
    # Save the prediction image
    cv2.imwrite(config['output_path'], mask_binary)
    print(f"✅ Prediction saved to: {config['output_path']}")

📂 Current working directory: c:\Users\hp\Desktop\image_Segmentation\Medical-Image-Segmentation-Deep-Learning-Project\Medical Image segmentation Notebook
